In [6]:
using DelimitedFiles
using Statistics
using StatsBase
using StatGeochem # for nanmean
using MAT
using Plots; gr();

In [15]:
res_path = "../data/base_single/base_large/"

"../data/base_single/base_large/"

In [99]:
dat, h = readdlm("$res_path/results-middle.csv", ',', header=true)

([2919.0 6.597149999999999 … 550.0 3.0; 2913.66 6.49955 … 550.0 4.0; … ; 2911.04 6.52915 … 550.0 2.0; 2862.64 6.7296499999999995 … 550.0 1.0], AbstractString["sample_rho" "sample_vp" … "formation_temp" "bin"])

In [100]:
ree_start = findfirst(isequal("MnO"), h[:])
ree_end = findfirst(isequal("Lu"), h[:])

58

In [101]:
trace_elts = "MnO P2O5 Li Be B F Cl Sc V Cr Co Ni Cu Zn Ga Rb Sr Y Zr Nb Mo Pd La Ce Pr Nd Sm Eu Gd Tb Dy Ho Er Tm Yb Lu"
trace_elts = split(trace_elts, " ");

In [102]:
ree_index = [i in trace_elts for i in h][:];

In [103]:
build_args = readdlm("$res_path/inversion_options.csv", ',', header=false)
N = build_args[findfirst(isequal("num_invert"), build_args[:,1]),2]
M = build_args[findfirst(isequal("num_runs"), build_args[:,1]),2]

## We'll actually use 252 for N because that's what we've decided is right 
N

252

In [104]:


runs_means = zeros(floor(Int, M*N/252), length(trace_elts))

for i in 1:size(runs_means, 1)
    start_run = (i-1)*252 + 1
    end_run = i*252
    runs_means[i,:] .= nanmean(dat[start_run:end_run, ree_index], dims=1)[:]
end

In [105]:
println(size(dat,1))
percent_good = sum(isnan.(dat[:, ree_index]), dims=1)./size(dat,1)
println("& Fraction defined & ", join(round.(percent_good, digits=3)[1:13], " & "), " \\\\")
println("& Fraction defined & ", join(round.(percent_good, digits=3)[14:26], " & "), " \\\\")
println("& Fraction defined & ", join(round.(percent_good, digits=3)[27:end], " & "), " \\\\")

2520000
& Fraction defined & 0.043 & 0.05 & 0.841 & 0.848 & 0.943 & 0.924 & 0.922 & 0.313 & 0.346 & 0.171 & 0.367 & 0.199 & 0.456 \\
& Fraction defined & 0.431 & 0.625 & 0.186 & 0.064 & 0.12 & 0.101 & 0.234 & 0.904 & 0.972 & 0.128 & 0.166 & 0.688 & 0.24 \\
& Fraction defined & 0.286 & 0.287 & 0.501 & 0.39 & 0.628 & 0.673 & 0.646 & 0.651 & 0.209 & 0.359 \\


In [106]:
# Compare to raw data
raw_dat = matread("../resources/igncn1.mat")
for name in trace_elts
    println(name, ": ", sum(isnan.(raw_dat[name]))/length(raw_dat[name]))
end

MnO: 0.0566699662280191
P2O5: 0.06588447653429604
Li: 0.8149237219052056
Be: 0.7824327471759637
B: 0.9395161290322581
F: 0.9099219750786072
Cl: 0.9180738325375568
Sc: 0.2675992779783393
V: 0.3394666356119716
Cr: 0.1713345755211366
Co: 0.3358856410853616
Ni: 0.2282520088505881
Cu: 0.42169849772912543
Zn: 0.3875189239548154
Ga: 0.5743420286479562
Rb: 0.2071881914521952
Sr: 0.04455863514615116
Y: 0.08288692209153371
Zr: 0.09301851636194247
Nb: 0.20116163968790032
Mo: 0.8872860137417026
Pd: 0.975748224059625
La: 0.07982997554442763
Ce: 0.12558227553278212
Pr: 0.6916851053918714
Nd: 0.20784325142657506
Sm: 0.27802201001513915
Eu: 0.26352334924886456
Gd: 0.4985879818330034
Tb: 0.37366076627460115
Dy: 0.6254512635379061
Ho: 0.6812914871317107
Er: 0.6539682077559101
Tm: 0.6472283684639571
Yb: 0.13385058809828812
Lu: 0.34427040875742404


In [107]:
# Compare to rudnick and gao 
rg , hr = readdlm("../resources/rudnick_gao_2014_res.csv", ',', header=true)
for (l, layer) in enumerate(["upper", "middle", "lower"])
    println()
    println(layer)
    print(" & Rudnick and Gao")
    for elt in trace_elts[1:13]
        idx = findfirst(isequal(elt), rg[:,1])
        print(" & ", rg[idx, l+1])
    end
    print(" \\\\")
    println()
    print(" & Rudnick and Gao")
    for elt in trace_elts[14:26]
        idx = findfirst(isequal(elt), rg[:,1])
        print(" & ", rg[idx, l+1])
    end
    print(" \\\\")
    println()
    print(" & Rudnick and Gao")
    for elt in trace_elts[27:end]
        idx = findfirst(isequal(elt), rg[:,1])
        print(" & ", rg[idx, l+1])
    end
    print(" \\\\")
    println()
end


upper
 & Rudnick and Gao & 0.1 & 0.15 & 24 & 2.1 & 17 & 557 & 370 & 14.0 & 97 & 92 & 17.3 & 47 & 28 \\
 & Rudnick and Gao & 67 & 17.5 & 84 & 320 & 21 & 193 & 12 & 1.1 & 0.52 & 31 & 63 & 7.1 & 27 \\
 & Rudnick and Gao & 4.7 & 1.0 & 4.0 & 0.7 & 3.9 & 0.83 & 2.3 & 0.3 & 2.0 & 0.31 \\

middle
 & Rudnick and Gao & 0.1 & 0.15 & 12 & 2.3 & 17 & 524 & 182 & 19 & 107 & 76 & 22 & 33.5 & 26 \\
 & Rudnick and Gao & 69.5 & 17.5 & 65 & 282 & 20 & 149 & 10 & 0.6 & 0.76 & 24 & 53 & 5.8 & 25 \\
 & Rudnick and Gao & 4.6 & 1.4 & 4.0 & 0.7 & 3.8 & 0.82 & 2.3 & 0.32 & 2.2 & 0.4 \\

lower
 & Rudnick and Gao & 0.1 & 0.1 & 13 & 1.4 & 2 & 570 & 250 & 31 & 196 & 215 & 38 & 88 & 26 \\
 & Rudnick and Gao & 78 & 13 & 11 & 348 & 16 & 68 & 5 & 0.6 & 2.8 & 8 & 20 & 2.4 & 11 \\
 & Rudnick and Gao & 2.8 & 1.1 & 3.1 & 0.48 & 3.1 & 0.68 & 1.9 & 0.24 & 1.5 & 0.25 \\


# Latex table formatting 

In [108]:

println(" & ", join(trace_elts[1:12], " & "), " \\\\")
println("\\hline")

# Mean of means
println("Middle & Median & ", join(round.(median(runs_means, dims = 1), digits=3)[1:12], " & "), " \\\\")

# 5th percentile 
print("& 5th percentile & ")
for j in 1:12
    ok = .! isnan.(runs_means[:,j])
    print(round(percentile(runs_means[ok,j], 5), digits=3), " & ")
end
print("\\\\")

println()

# 95th percentile 
print("& 95th percentile & ")
for j in 1:12
    ok = .! isnan.(runs_means[:,j])
    print(round(percentile(runs_means[ok,j], 95), digits=3), " & ")
end
print("\\\\")



 & MnO & P2O5 & Li & Be & B & F & Cl & Sc & V & Cr & Co & Ni \\
\hline
Middle & Median & 0.14 & 0.257 & 20.902 & 3.175 & 18.496 & 9.817 & 28.304 & 22.165 & 174.119 & 166.51 & 30.35 & 76.218 \\
& 5th percentile & 0.132 & 0.232 & 15.72 & 2.314 & 10.767 & 0.068 & 6.171 & 20.484 & 159.928 & 136.129 & 27.589 & 63.728 & \\
& 95th percentile & 0.158 & 0.285 & 32.432 & 4.667 & 46.791 & 246.708 & 121.366 & 23.962 & 188.972 & 205.433 & 33.262 & 96.225 & \\

In [109]:
println(" & ", join(trace_elts[13:23], " & "), " \\\\")
println("\\hline")

# Mean of means
println("Middle & Median & ", join(round.(nanmedian(runs_means, dims = 1), digits=3)[13:23], " & "), " \\\\")

# 5th percentile 
print("& 5th percentile & ")
for j in 13:23
    ok = .! isnan.(runs_means[:,j])
    print(round(percentile(runs_means[ok,j], 5), digits=3), " & ")
end
print("\\\\")

println()

# 95th percentile 
print("& 95th percentile & ")
for j in 13:23
    ok = .! isnan.(runs_means[:,j])
    print(round(percentile(runs_means[ok,j], 95), digits=3), " & ")
end
print("\\\\")

 & Cu & Zn & Ga & Rb & Sr & Y & Zr & Nb & Mo & Pd & La \\
\hline
Middle & Median & 63.657 & 88.114 & 20.637 & 64.634 & 475.458 & 26.166 & 181.041 & 18.461 & 4.402 & 5.346 & 34.059 \\
& 5th percentile & 50.83 & 79.754 & 19.31 & 55.27 & 426.154 & 24.082 & 161.829 & 14.727 & 2.488 & 0.929 & 29.591 & \\
& 95th percentile & 128.229 & 111.93 & 26.367 & 75.641 & 543.749 & 30.31 & 204.203 & 23.792 & 32.469 & 242.031 & 39.565 & \\

In [110]:
println(" & ", join(trace_elts[(24):size(runs_means,2)], " & "), " \\\\")
println("\\hline")

# Mean of means
println("Middle & Median & ", join(round.(nanmedian(runs_means, dims = 1), digits=3)[(24):size(runs_means,2)], " & "), " \\\\")

# 5th percentile 
print("& 5th percentile & ")
for j in (24):size(runs_means,2)#size(runs_means,2)
    ok = .! isnan.(runs_means[:,j])
    print(round(percentile(runs_means[ok,j], 5), digits=3), " & ")
end
print("\\\\")

println()

# 95th percentile 
print("& 95th percentile & ")
for j in (24):size(runs_means,2) #size(runs_means,2)
    ok = .! isnan.(runs_means[:,j])
    print(round(percentile(runs_means[ok,j], 95), digits=3), " & ")
end
print("\\\\")

#println(join(round.(percentile.(runs_means, 95), digits=2), " & "))

 & Ce & Pr & Nd & Sm & Eu & Gd & Tb & Dy & Ho & Er & Tm & Yb & Lu \\
\hline
Middle & Median & 66.533 & 7.106 & 30.649 & 5.768 & 1.534 & 5.242 & 0.8 & 4.574 & 0.927 & 2.522 & 0.38 & 2.528 & 0.363 \\
& 5th percentile & 58.119 & 5.668 & 27.064 & 5.185 & 1.409 & 4.697 & 0.725 & 4.072 & 0.818 & 2.242 & 0.336 & 2.306 & 0.329 & \\
& 95th percentile & 76.821 & 9.155 & 35.152 & 6.603 & 1.679 & 6.075 & 0.973 & 5.264 & 1.164 & 2.962 & 0.441 & 2.86 & 0.579 & \\

13.0

In [148]:
println(join(h[ree_start:ree_end], " & "))

MnO & P2O5 & Li & Be & B & F & S & Cl & Sc & V & Cr & Co & Ni & Cu & Zn & Ga & As & Se & Rb & Sr & Y & Zr & Nb & Mo & Pd & La & Ce & Pr & Nd & Sm & Eu & Gd & Tb & Dy & Ho & Er & Tm & Yb & Lu


In [ ]:
wt \% & wt \% & PPM & PPM & PPM & PPM & PPM & PPM & PPM & PPM & PPM & PPM & PPM 
PPM & PPM & PPM & PPM & PPM & PPM & PPM & PPM & PPM & PPM & PPM & PPB & PPM 
PPM & PPM & PPM & PPM & PPM & PPM & PPM & PPM & PPM & PPM & PPM & PPM & PPM
